## *Machine Transliteration of Named Entities from Hindi to English*
***(using Conditional Random Fields and Neural networks)***

***Author : Dhawal Darji***

### Libraries & Resources

In [48]:
# uncomment the following lines when you run this for the first time. 
# It will install necessary libraries into your python environment.
# You only need to run it once, then you can comment it out again to save some time.

# !pip install indic-transliteration
# !pip install sklearn_crfsuite

#!conda install --yes --prefix {sys.prefix} numpy
#!conda install --yes --prefix {sys.prefix} scipy
#!conda install --yes --prefix {sys.prefix} pandas

import os

# Custom utils

'''Syllabifier takes input as a hindi-english word pair and returns syllabified hindi-english words'''
import syllabifier as sb

'''dataLoader contains utility functions to load/split/manipulate data'''
import dataLoader as dl

# Libraries
import numpy as np
import regex as re
from sklearn_crfsuite import CRF


### Loading Data

In [126]:
# loads data from a txt file into a numpy array

data = dl.loadTxt('datasetBig.txt')

Size of dataset: 45201
One entry in dataset: विठ्ठल,vitthal


### Syllabification

In [133]:
# Syllabifies the data into hindiSyllabifiedWord, englishSyllabifiedWord

data = np.array([d for d in data if ('\u093c' not in d.split(",")[0] and 
                                     '\u0949' not in d.split(",")[0] and 
                                     '\u0911' not in d.split(",")[0] and 
                                    '\u0945' not in d.split(",")[0])])

sylData = np.array([sb.syllabify(d.split(",")[0], d.split(",")[1]) for d in data])
print("One entry in Syllabified data:\n", sylData[0])



42935
One entry in Syllabified data:
 [['वि' 'vi']
 ['ठ्ठ' 'ththa']
 ['ल' 'l']]


### Utilities & Helper functions

In [134]:
# List of hindi vowels
hVowels = ['\u0905','\u0906','\u0907','\u0908','\u0909','\u090A','\u090B','\u090F','\u0910','\u0913','\u0914']

# List of hindi consonants
hConsonants = [u'\u0915',u'\u0916',u'\u0917',u'\u0918',u'\u0919',u'\u091A',u'\u091B',u'\u091C',
               u'\u091D',u'\u091E',u'\u091F',u'\u0920',u'\u0921',u'\u0922',u'\u0923',u'\u0924',
              u'\u0925',u'\u0926',u'\u0927',u'\u0928',u'\u092A',u'\u092B',
               u'\u092C',u'\u092D',u'\u092E',u'\u092F',u'\u0930',u'\u0932',u'\u0933',u'\u0935',
              u'\u0936',u'\u0937',u'\u0938',u'\u0939']

# List of hindi consonants in english
eConsonants = 'ka,kha,ga,gha,cha,ja,jha,ta,tha,da,dha,na,pa,pha,ba,bha,ma,ya,ra,la,va,sha,sa,ha'.split(",")

def wordLen(word):
    return range(len(word))

def getHindi(word):
    '''Returns the Hindi(X) words from data'''
    return np.array(word).T[0].T

def getEnglish(word):
    '''Returns the English(Y) words from data'''
    return np.array(word).T[1].T

### Evaluation metrics

In [135]:
def elem_accuracy(y_hat_l, y_gold):
    '''Evaluation measure for predictions of a y-sequence, returns average position-wise matches.'''
    y_hat = np.array(y_hat_l)
    if len(y_hat) != len(y_gold):
        raise ValueError(len(y_hat),' != ',len(y_gold))
    matches = np.sum(y_gold == np.array(y_hat))
    return 1.0 * matches / len(y_gold)

### Train/Test Split

In [136]:
# Splits the syllabified data into train/test/validation test

import collections

# count the number of occurrences of a syllable
countList = []
for d in sylData:
    for s in getEnglish(d):
        countList.append(s)

countDict = collections.Counter(countList)

# select the most frequent syllables as tags
tags = [key for key,count in countDict.items() if count > 1]
print("All tags:", tags)
print("Number of tags:", len(tags))

# syllables other than the tags will be marked as 'x'
for d in sylData:
    for i, s in enumerate(d):
        if s[1] not in tags:
            d[i][1] = "x"

# Split the data into train/validation/test set
# Validation set is 15% of the train set
trainData, valData, testData = dl.splitData(sylData, 80) #(syllabifiedData, trainSize)

    

All tags: ['vi', 'ththa', 'l', 'ma', 'ni', 'ka', 'ra', 'v', 'shan', 'ta', 'nu', 'jyo', 'ti', 'an', 'ba', 'r', 'a', 'nan', 'ha', 's', 'la', 'kshma', 'n', 'san', 'pa', 'da', 'va', 'rdha', 'pra', 'sa', 'd', 'shu', 'to', 'he', 'u', 'rda', 'le', 'k', 'e', 'na', 't', 'j~na', 'ne', 'shva', 'go', 'dau', 'jen', 'ghu', 'su', 're', 'no', 'je', 'mi', 'di', 'sha', 'ru', 'chi', 'dha', 'dhu', 'sham', 'm', 'rna', 'mo', 'rcha', 'pu', 'rva', 'ju', 'chan', 'dra', 'kam', 'yo', 'gi', 'rmi', 'bha', 'chai', 'li', 'tra', 'vai', 'y', 'ga', 'ri', 'hu', 'rya', 'tha', 'gha', 'ja', 'man', 'vin', 'tu', 'so', 'ya', 'pri', 'vrri', 'b', 'smi', 'dhi', 'bhi', 'ke', 'me', 'bra', 'mha', 'in', 'rma', 'de', 'tya', 'o', 'on', 'krri', 'shna', 'shri', 'rri', 'shi', 'ran', 'g', 'ai', 'ji', 'pha', 'gho', 'rsha', 'hen', 'sin', 'ro', 'hi', 'gau', 'du', 'pi', 'lu', 'tna', 'sai', 'p', 'j', 'rmen', 'ren', 'ska', 'nyu', 'rju', 'ku', 'yu', 'shthi', 'si', 'ryo', 'dhrri', 'dro', 'khmi', 'pam', 'vya', 'c', 'tru', 'lmi', 'gri', 'kra', 'cha

In [137]:
# Getting the X and Y from data

trainX = np.array([getHindi(data) for data in trainData])
trainY = np.array([getEnglish(data) for data in trainData])

valX = np.array([getHindi(data) for data in valData])
valY = np.array([getEnglish(data) for data in valData])

testX = np.array([getHindi(data) for data in testData])
testY = np.array([getEnglish(data) for data in testData])

## Sklearn CRF

### Feature generation

In [266]:
def syllable2features(word, index):
    '''Takes a syllable as input and returns a feature vector'''
    
    hindiSyllable = getHindi(word)[index]
    engSyllable = getEnglish(word)[index]

    features = {
        'intercept': 1,
        'hindi': hindiSyllable,
        'isVowel': 1 if hindiSyllable in hVowels else 0,
        'isConsonant': 1 if hindiSyllable in hConsonants else 0,
        'containsMatra': 1 if hindiSyllable not in hConsonants else 0,
        'containsHalfConsonant': 1 if u'\u094D' in hindiSyllable else 0,  
    }
    
    if index > 0:
        prevHindiSyllable = getHindi(word)[index-1]
        prevEngSyllable = getEnglish(word)[index-1]
        features.update({
            'prevHindiSyllable': prevHindiSyllable,
            'prevEngSyllable': prevEngSyllable,
            'prevVowel': 1 if prevHindiSyllable in hVowels else 0,
        })
    
    else:
        features['BOW'] = 1
        
    if index < len(word)-1:
        nextHindiSyllable = getHindi(word)[index+1]
        nextEngSyllable = getEnglish(word)[index+1]
        features.update({
            'nextHindiSyllable': nextHindiSyllable,
            'nextEngSyllable': nextEngSyllable,
            'nextVowel': 1 if nextHindiSyllable in hVowels else 0,
        })
    else:
        features['EOW'] = 1

        
    return features

def word2features(word):
    '''takes a whole word as input and returns feature vector for each syllable'''
    return np.array([syllable2features(word, index) for index in range(len(word))])

# Get the training, validation and test feature vector
TrainX = np.array([word2features(w) for w in trainData])
ValX = np.array([word2features(w) for w in valData])
TestX = np.array([word2features(w) for w in testData])

print(word2features([['न' , 'na'],
              ['म', 'ma'], 
              ['स्ते', 'ste']]))

[{'intercept': 1, 'hindi': 'न', 'isVowel': 0, 'isConsonant': 1, 'containsMatra': 0, 'containsHalfConsonant': 0, 'BOW': 1, 'nextHindiSyllable': 'म', 'nextEngSyllable': 'ma', 'nextVowel': 0}
 {'intercept': 1, 'hindi': 'म', 'isVowel': 0, 'isConsonant': 1, 'containsMatra': 0, 'containsHalfConsonant': 0, 'prevHindiSyllable': 'न', 'prevEngSyllable': 'na', 'prevVowel': 0, 'nextHindiSyllable': 'स्ते', 'nextEngSyllable': 'ste', 'nextVowel': 0}
 {'intercept': 1, 'hindi': 'स्ते', 'isVowel': 0, 'isConsonant': 0, 'containsMatra': 1, 'containsHalfConsonant': 1, 'prevHindiSyllable': 'म', 'prevEngSyllable': 'ma', 'prevVowel': 0, 'EOW': 1}]


### Training

In [139]:
from sklearn_crfsuite import CRF, metrics
from datetime import datetime

# The crf model with the hyperparameters
crf = CRF(
algorithm='lbfgs',
c2=0.1,
c1=1.0,
max_iterations=100,
all_possible_transitions = True)

# training!
start = datetime.now()
crf.fit(TrainX, trainY)
end = datetime.now()

### Evaluation

In [142]:
labels = list(crf.classes_)

# predict on train, val and test sets
ypredT = crf.predict(TrainX)
ypredV = crf.predict(ValX)
ypred = crf.predict(TestX)

In [251]:
print("Training time:", (end - start))

print("\nTrain Accuracy:", np.average([elem_accuracy(ypredT[i], trainY[i]) for i in range(len(trainY))]))
print("Train F1 score:", metrics.flat_f1_score(ypredT, trainY, average='weighted'))

print("\nValidation Accuracy:", np.average([elem_accuracy(ypredV[i], valY[i]) for i in range(len(valY))]))
print("Validation F1 score:", metrics.flat_f1_score(ypredV, valY, average='weighted'))

print("\nTest Accuracy:", np.average([elem_accuracy(ypred[i], testY[i]) for i in range(len(testY))]))
print("Test F1 score:", metrics.flat_f1_score(ypred, testY, average='weighted'))

Training time: 8:43:55.616911

Train Accuracy: 0.9936271711497769
Train F1 score: 0.9957018132202916

Validation Accuracy: 0.9871894409937888
Validation F1 score: 0.9906058358129142

Test Accuracy: 0.98552920527708
Test F1 score: 0.9887670197817535


## Custom CRF

This part is potentially broken and doesn't work completely

### Helper methods

In [24]:
X = 0
Y = 1

def ks(seq):
    '''Returns the range over a seq'''
    return range(1, len(seq))

def log_factor(w, feats):
    '''Returns the factor in log space'''
    return np.dot(w, feats)

def factor(w, feats):
    '''Returns the factor in probability space '''
    return np.exp(log_factor(w, feats))

def general_features(tagPrev, x_seq, k, ftype="uni"):
    '''Return features for a given sequence'''
    return features(tagPrev,x_seq[k-1],k,ftype)

def general_factor(w,tagPrev, x_seq, k):
    '''Return factor for a given sequence'''
    return factor(w, general_features(tagPrev, x_seq, k))

print(general_factor(np.ones(maxUnif),'ap',trainX[1], 1))
print(general_features('ka',trainX[0], 1))

20.085536923187668
[1. 0. 1. 0. 0. 1.]


### Features Generation

In [10]:
# Unigram features
def unigram(tagPrev, word, index):
    hindiSyllable = word #[index]
    f1 = 1.0
    f2 = 1.0 if hindiSyllable in hVowels else 0.0
    f3 = 1.0 if hindiSyllable in hConsonants else 0.0
    f4 = 1.0 if hindiSyllable not in hConsonants else 0.0 # is the syllable a complete consonant?
    f5 = 1.0 if u'\u094d' in hindiSyllable else 0.0 # does the syllable contain a half consonant?

    return np.array([f1, f2, f3, f4, f5])
maxUnif = 5

# Bigram features
def bigram(word, index):
    hindiSyllable = word[index]
    
    f1 = 1.0
    f2 = 1.0 if hindiSyllable in hVowels else 0.0
    f3 = 1.0 if hindiSyllable in hConsonants else 0.0
    f4 = 1.0 if hindiSyllable not in hConsonants else 0.0 # is the syllable a complete consonant?
    f5 = 1.0 if u'\u094d' in hindiSyllable else 0.0 # does the syllable contain a half consonant?
    f6 = 1.0 if index > 0 else 0.0 # if index > 0, currY will also depend on prevX, prevY
        
    return np.array([f1, f2, f3, f4, f5, f6])
maxBif = 6

# Trigram features
def trigram(word, index):
    hindiSyllable = word[index]
    f1 = 1.0
    f2 = 1.0 if hindiSyllable in hVowels else 0.0
    f3 = 1.0 if hindiSyllable in hConsonants else 0.0
    f4 = 1.0 if hindiSyllable not in hConsonants else 0.0 # is the syllable a complete consonant?
    f5 = 1.0 if u'\u094d' in hindiSyllable else 0.0 # does the syllable contain a half consonant?
    f6 = 1.0 if index > 0 else 0.0 # if index > 0, currY will also depend on prevX, prevY
    f7 = 1.0 if index < len(word)-1 else 0 # currY will depend on both prevX,prevY and prev-1X, prev-1Y
    
    return np.array([f1, f2, f3, f4, f5, f6, f7])
maxTrif = 7

# Generates features with given feature model
def features(tagPrev, word, index, ftype = "uni"):
    
    if ftype == "uni":
        return unigram(tagPrev, word, index)
    
    if ftype == "bi":
        return bigram(word, index)
    
    if ftype == "tri":
        return trigram(word, index)
    
    

### Generating a feature & factor matrix

In [107]:
def word2features(word):
#     return np.array([syllable2features(word, index) for index in range(len(word))])
    return np.array([general_features("",word, index, "uni") for index in wordLen(word)])

TrainX = np.array([word2features(w) for w in trainX])


def getFeatureMatrix(ftype = "uni"):
    '''Creates a feature matrix over the whole data with given feature model'''
    return np.array([
    np.array([general_features(trainY[i][k-1], trainX[i], k)
    for k in ks(trainX[i])])
    for i in range(len(trainData))])

featureMatrix = getFeatureMatrix()
print(featureMatrix[0])

def Fj(word):
    '''F(x,y) feature statistics along the chain; returns an array indexed by feature_idx j'''
    return np.sum([general_features(getEnglish(word)[index-1], getHindi(word), index, "uni") for index in ks(word)], axis=0)

factorMatrix = []
def getFactorMatrix(w):
    '''Returns a factor matrix over the whole data with given weight vector'''
    factMat = np.array([
        np.array([general_factor(w,trainY[i][k-1], trainX[i], k)
         for k in ks(trainX[i])])
        for i in range(len(trainData))
    ])
    return factMat

# Create weight vector according to feature model
w0 = np.ones(maxUnif)

factorMatrix = getFactorMatrix(w0)
print(factorMatrix[0])

[[1. 0. 1. 0. 0. 1.]
 [1. 0. 1. 0. 0. 1.]
 [1. 0. 0. 1. 0. 0.]]
[20.08553692 20.08553692  7.3890561 ]
(36160,)


### Alphas/Betas

In [115]:
tagIndex = {tag: tagId for tagId, tag in enumerate(tags)}

def alpha_trellis(w, word):
    ''' returns a dictionary of forward variables {k : array of length num_states}'''    
    alphas = {0: np.array([1 for id in tagIndex])}
    factorMat = getFactorMatrix(w)
    
    for k in ks(word):
        alphas[k] = np.einsum('i,j->i',alphas[k-1],factorMat[k-1])
#         alphas[k] = np.array([
#             np.sum([alphas[k-1][sPrevIdx] * general_factor(w, tagPrev,getHindi(word), k)
#             for sPrev, sPrevIdx in tagIndex.items()])
#             for tagPrev in tags
#         ])
    kmax = len(word)
    alphas[kmax] = np.sum(alphas[kmax-1])
    return alphas

def beta_trellis(w, word):
    ''' returns a dictionary of backward variables {k : array of length num_states}'''
    kmax = len(word)
    betas = {kmax: np.array([1 for tag in tagIndex])}
    factorMat = getFactorMatrix(w)
    for k in reversed(ks(word)):
        betas[k] = np.einsum('i,j->i',betas[k+1],factorMat[k+1])
#         betas[k] = np.array([
#                 np.sum([ betas[k+1][sNextIdx] * general_factor(w, tagPrev, getHindi(word),k)
#                     for sNext, sNextIdx in tagIndex.items()])
#                 for tagPrev in tags])

    betas[0] = np.sum(betas[1])
    return betas

w0 = np.ones(maxUnif)
print(alpha_trellis(w0, trainData[0]))
print(beta_trellis(w0, trainData[0]))

{4: array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1]), 3: array([40.17107385, 40.17107385, 40.17107385, 40.17107385, 40.17107385,
       40.17107385, 40.17107385, 40.17107385, 40.17107385, 40.17107385,
       40.17107385, 40.17107385, 40.17107385, 40.17107385, 40.17107385,
       40.17107385, 40.17107385, 40.17107385, 40.17107385, 40.17107385,
       40.17107385, 40.17107385, 40.17107385, 40.17107385]), 2: array([593.65263641, 593.65263641, 593.65263641, 593.65263641,
       593.65263641, 593.65263641, 593.65263641, 593.65263641,
       593.65263641, 593.65263641, 593.65263641, 593.65263641,
       593.65263641, 593.65263641, 593.65263641, 593.65263641,
       593.65263641, 593.65263641, 593.65263641, 593.65263641,
       593.65263641, 593.65263641, 593.65263641, 593.65263641]), 1: array([40158.02847821, 40158.02847821, 40158.02847821, 40158.02847821,
       40158.02847821, 40158.02847821, 40158.02847821, 40158.02847821,
       40158.02847821, 40158

### Gradient Calculations & Optimizations

In [116]:
from numpy import newaxis

w0 = np.ones(maxUnif)

def fb_expectation_onseq(w, seq, alphas, betas):
    '''part B of grad(w) for one sequence seq=(x,y) - returns a scalar'''
    
    return np.sum([
        np.sum([
            general_features(sPrev, getHindi(seq), k, ftype="uni") *
            alphas[k-1][sPrevIdx] * 
             general_factor(w, sPrev, getHindi(seq), k) *
             betas[k+1][sNextIdx]
        for sNext, sNextIdx in tagIndex.items()
        for sPrev, sPrevIdx in tagIndex.items()], axis = 0)
    for k in ks(seq)], axis = 0) / betas[0]

alphas = alpha_trellis(w0, trainData[0])
betas = beta_trellis(w0, trainData[0])

fb_expect = fb_expectation_onseq(w0, trainData[0], alphas, betas)
fb_expect

array([31.68327258,  0.        , 22.93449075,  8.74878183,  0.        ,
       25.95441513])

In [122]:
def gradient_A(seqs):
    '''part A (empirical) of grad(w) seqs=[(x1,y1), (x2,y2) ...]'''
    return np.sum([Fj(s) for s in seqs])

def gradient_B_oneseq(w,seq):
    return fb_expectation_onseq(w, seq, alpha_trellis(w, seq), beta_trellis(w, seq))

def gradient_B(w, seqs):
    '''part B (expectation) of grad(w) seqs=[(x1,y1), (x2,y2) ...] '''
    
    # b is the sum of all the expectations
    return np.sum([gradient_B_oneseq(w, s) for s in seqs], axis = 0)

# def gradient_B_oneseq(w, seq):
#     alphas = alpha_trellis(w, seq)
#     betas = beta_trellis(w, seq)
    
#     features = featureMatrix
#     factors = getFactorMatrix(w)
#     i = 0
#     return np.sum(np.array([
#         np.einsum('i,i,j,k->i', alphas[k-1], betas[k+1], features[k][i], factors[k]) for k in ks(seq)
#     ]), axis = 0)

# def gradient_B(w, seqs):
#     return np.sum([gradient_B_oneseq(w, seq) for seq in seqs])

def gradient_C(w, sigmasq):
    '''part C (regularizer) of grad(w)  '''
    # c = w/sigma^2
    return w / sigmasq


def Z(w, seq):
    # The first beta will be the last and final Z for the chain
    return beta_trellis(w, seq)[0]


def gradient(w, seqs, sigmasq):
    ''' Gradient of the model at w. sigmasq is the inverse strength of the regularizer'''
    return gradient_A(seqs) - gradient_B(w, seqs) - gradient_C(w, sigmasq)

def log_likelihood(w, seqs, sigmasq):
    '''Log-likelihood of the model at w. Sigmasq is the inverse strength of the regularizer'''
    
    # a is the sum of all the dot-products of w & fj(seq)
    a = np.sum([np.dot(w, Fj(s)) for s in seqs])
    
    # b is the sum of log(Z) for every sequence
    
    b = np.sum([np.log(Z(w, s)) for s in seqs])
    
    # c is the sum of weight's square by twice the sigma square
    c = np.sum(w**2 / 2 * sigmasq)
    
    return a - b - c

# gradient_B(w0, trainData)

In [123]:
gradient_A(trainData) , gradient_B(w0, trainData[:2]), gradient_C(w0, 1.0), Z(w0,trainData[0])

(array([87742.,  4628., 28334., 59408.,  7642., 37001.]),
 array([46.46237269, 14.77910011, 22.93449075, 23.52788194,  0.        ,
        38.06121056]),
 array([1., 1., 1., 1., 1., 1.]),
 963792.6834771498)

In [124]:
# Self test!
import scipy.optimize

print ('divergence from gradient(w0)',scipy.optimize.check_grad(log_likelihood, gradient,  w0, trainData[0:2], 1.0))

def neg_logl(w, seqs, sigmasq):
    return - log_likelihood(w, seqs, sigmasq)

def neg_grad(w, seqs, sigmasq):
    return - gradient(w, seqs, sigmasq)


divergence from gradient(w0) 48.907192549220575


In [125]:
def estimate_w(w, seqs, sigmasq, call_back=None):
    return scipy.optimize.minimize( fun=neg_logl, x0=w, jac=neg_grad, args=(seqs, sigmasq), callback=call_back, method='TNC')

In [126]:
# initial likelihood
print ('w0 logl', log_likelihood(w0, trainData[:2], 1.0))

w0 logl -13.170968622541302


In [128]:
# more self-tests

def estim_debug(w):
    print (log_likelihood(w, trainData[:2], 1.0), 'weights', w)


res = estimate_w(w0, trainData[:2], 1.0 , estim_debug)
w_hat = res.x

print ('w_hat', w_hat)

-11.848019335915831 weights [-0.41066185  0.81103117  0.78062515  0.33570233  1.52698933  0.18878533]
w_hat [-0.41066185  0.81103117  0.78062515  0.33570233  1.52698933  0.18878533]


In [129]:
# estimated likelihood
print ('w_hat logl', log_likelihood(w_hat, trainData[0:2], 1.0))

print ('w_hat grad', neg_grad(w_hat, trainData[0:2], 1.0))

w_hat logl -11.848019335915831
w_hat grad [26.35455164 -1.01839324 11.30365041  5.57789055 -9.47301067 14.4197815 ]


### Decoding Predictions (Deltas)

In [131]:
def delta_trellis(w,seq):
    ''' Compute the trellis with delta (max-scores) and phi (max-configurations) variables '''

    deltas = {0: np.array([1.0 for s in tags])}
    phis = {0: np.array([tagId for tag, tagId in tagIndex.items()], dtype = int)}
    size = len(tags)
#     scores = []
    
    factorMat = getFactorMatrix(w)
    
    for k in ks(seq):
        # let initial deltas and phis of k be zeros
        deltas[k] = np.zeros(size)
        phis[k] = np.zeros(size, dtype = int)
        
        for sNext, sNextIdx in tagIndex.items():
            
#             sScore = np.array([
#                 deltas[k-1][sPrevIdx] * factorMat[k]
#                 for sPrev, sPrevIdx in tagIndex.items()
#             ])

            sScore = np.array([
                deltas[k-1][sPrevIdx] * general_factor(w, sPrev, getHindi(seq), k)
                for sPrev, sPrevIdx in tagIndex.items()
            ])
        
            # get the maxScore
            maxScore = np.argmax(sScore)

            deltas[k][sNextIdx] = sScore[maxScore]
            phis[k][sNextIdx] = maxScore

    return deltas, phis

def decode_trellis(seq, deltas, phis):
    '''decodes the trellis and returns (y_hat, prob) where y_hat is the most likely sequence and prob is its unnormalized probability'''
    
    maxk = len(seq) - 1
    endBest = np.argmax(deltas[maxk])
    
    yhat = np.zeros(len(seq), dtype = int)
    yhat[maxk] = endBest
    yhat_score = deltas[maxk][endBest]
    
    # go in reverse
    for k in reversed(ks(seq)):
        yhat[k-1] = phis[k][yhat[k]]
        
    return yhat, yhat_score


def predict(w, seq):
    ''' Predicts the most likely sequence and returns its (unnormalized) probability.'''

    deltas, phis = delta_trellis(w, seq)
    yhat, yhat_prob = decode_trellis(seq, deltas, phis)

    return yhat, yhat_prob/Z(w,seq)


In [132]:
import pandas as pd

def print_trellis():
    deltas,phis = delta_trellis(w0, trainData[0])
    print ('w=', w0)
    print ('deltas')
    print (pd.DataFrame(deltas))
    print ('')
    print ('phis')
    print (pd.DataFrame(phis))

    
print_trellis()

w= [1. 1. 1. 1. 1. 1.]
deltas
      0          1           2            3
0   1.0  20.085537  403.428793  8103.083928
1   1.0  20.085537  403.428793  8103.083928
2   1.0  20.085537  403.428793  8103.083928
3   1.0  20.085537  403.428793  8103.083928
4   1.0  20.085537  403.428793  8103.083928
5   1.0  20.085537  403.428793  8103.083928
6   1.0  20.085537  403.428793  8103.083928
7   1.0  20.085537  403.428793  8103.083928
8   1.0  20.085537  403.428793  8103.083928
9   1.0  20.085537  403.428793  8103.083928
10  1.0  20.085537  403.428793  8103.083928
11  1.0  20.085537  403.428793  8103.083928
12  1.0  20.085537  403.428793  8103.083928
13  1.0  20.085537  403.428793  8103.083928
14  1.0  20.085537  403.428793  8103.083928
15  1.0  20.085537  403.428793  8103.083928
16  1.0  20.085537  403.428793  8103.083928
17  1.0  20.085537  403.428793  8103.083928
18  1.0  20.085537  403.428793  8103.083928
19  1.0  20.085537  403.428793  8103.083928
20  1.0  20.085537  403.428793  8103.083928
21

### Evaluation

In [ ]:
# compare performance on test data with w0 (one vector), w_hat (estimated) and w_rnd (random weights)

w_rnd = np.random.randn(maxUnif)
print ('w_rnd', w_rnd)

print ('x_testseq', getHindi(testData[0]))
y_rnd,p_rnd = predict(w_rnd, testData[0])
y_one,p_one = predict(w0, testData[0])
y_hat,p_hat = predict(w_hat, testData[0])


y_gold = getEnglish(testData[0])
print ('yrnd',np.array(y_rnd), elem_accuracy(y_rnd, y_gold ), p_rnd)
print ('yone',np.array(y_one), elem_accuracy(y_one, y_gold ), p_one)
print ('yhat',np.array(y_hat), elem_accuracy(y_hat, y_gold ), p_hat)
print ('y   ', y_gold)

print ('accuracy of w_one', np.average([ elem_accuracy(predict(w0, seq)[0], getEnglish(seq)) for seq in testData]))
print ('accuracy of w_hat', np.average([ elem_accuracy(predict(w_hat, seq)[0], getEnglish(seq)) for seq in testData]))

w_rnd [ 0.58669579  0.78890075 -0.71493533  1.35097147  1.37760701  0.35548147]
x_testseq ['अ' 'म' 'ज' 'द']


<ipython-input-130-017088822691>:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  matches = np.sum(y_gold == np.array(y_hat))


yrnd [1 1 1 0] 0.0 0.0008624228156494865
yone [1 1 1 0] 0.0 0.022853945845843236
yhat [1 1 1 0] 0.0 0.009773437632299398
y    ['a' 'ma' 'ja' 'x']
